In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyarabic

     |████████████████████████████████| 112kB 28.2MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=60a005258e5cb9e2bbd6745f1ac9bf5561adb71945c9027b569d22915e7ef3c6
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import time
import glob
from random import shuffle
from pyarabic import araby
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Done only once

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c machathon2-qualifications

     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=3193851114f399226e24de016e888d3cf0fd07318770224371928fa1c824544c
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
  0% 0.00/953k [00:00<?, ?B/s]
100% 953k/953k [00:00<00:00, 144MB/s]


In [ ]:
import zipfile
filename = "machathon2-qualifications.zip"
with zipfile.ZipFile(filename, 'r') as zipf:
  zipf.extractall()
  print("Done")

Done


In [ ]:
with open('labels.txt', 'r') as f:
  label2name = f.readlines()
  label2name = [name.replace('\n', '') for name in label2name]

In [ ]:
import pandas as pd

data = pd.read_csv('train.csv')
train_data = data[:19900]
test_data = data[19900:]

In [ ]:
x_train = np.array(train_data['data'])
y_train  = np.array(train_data['labels'])

x_test = np.array(test_data['data'])
y_test = np.array(test_data['labels'])

In [ ]:
execluded = '!()*-ـ.:=o[]«»;؛,،~?؟\u200f\ufeffـ'

for i in tqdm(range(len(x_train))):
  out = ""
  #x_train[i] = araby.strip_tashkeel(x_train[i])
  for c in x_train[i]:
    if c not in execluded:
      out += c
  x_train[i] = out

for i in tqdm(range(len(x_test))):
  out = ""
  #x_test[i] = araby.strip_tashkeel(x_test[i])
  for c in x_test[i]:
    if c not in execluded:
      out += c
  x_test[i] = out

100%|██████████| 100/100 [00:00<00:00, 31559.85it/s]


In [ ]:
X_train, X_valid , y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.1, random_state = 41)

In [ ]:
# Creating a mapping from unique characters to indices
vocab = set(' '.join(x_train))
char2idx = {u:i+1 for i, u in enumerate(vocab)}

def to_sequences(X):
  X = [[char2idx[char] for char in line] for line in X]
  X = pad_sequences(X, padding='post', value=0, maxlen = 100)
  return X
 
X_train = to_sequences(X_train)
X_valid = to_sequences(X_valid)

y_train = np.array(y_train)
y_valid = np.array(y_valid)

## My model

In [ ]:
## Steps taken to get the 95% accuracy:

### Training 40 epochs on the first Model, then realoading the same weights 
### following up with 20 epochs on the second model which only differs from
### the first in the dropout layers

from tensorflow import keras


############### 40 epochs #########

model = Sequential()
model.add(Input((100,)))
model.add(Embedding(len(char2idx)+1,256))
model.add(Bidirectional(tf.keras.layers.LSTM(units = 256, return_sequences=True)))
model.add(Dropout(0.7))##0.75 now
#model.add(Bidirectional(tf.keras.layers.LSTM(units = 512, return_sequences=True)))
#model.add(Dropout(0.75))
model.add(Bidirectional(tf.keras.layers.LSTM(units = 1024)))
model.add(Dropout(0.75)) ## 0.8 now
model.add(Dense((512), activation = 'relu'))
model.add(Dropout(0.75)) ## 0.8 now
model.add(Dense(len(label2name), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

##### + ########

############### 20 epochs #########

model = Sequential()
model.add(Input((100,)))
model.add(Embedding(len(char2idx)+1,256))
model.add(Bidirectional(tf.keras.layers.LSTM(units = 256, return_sequences=True)))
model.add(Dropout(0.75))##0.8 now
#model.add(Bidirectional(tf.keras.layers.LSTM(units = 512, return_sequences=True)))
#model.add(Dropout(0.75))
model.add(Bidirectional(tf.keras.layers.LSTM(units = 1024)))
model.add(Dropout(0.8)) ## 0.8 now
model.add(Dense((512), activation = 'relu'))
model.add(Dropout(0.8)) ## 0.85 now
model.add(Dense(len(label2name), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Model summary

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 256)          12032     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 100, 512)          1050624   
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 2048)              12591104  
_________________________________________________________________
dropout_11 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)              

## training

In [ ]:
## Creating a callback to recall best accuracy achieved on the validation set

callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
callbacks += [tf.keras.callbacks.ModelCheckpoint('full_verse.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

In [ ]:
model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 30, batch_size= 32, shuffle = True, callbacks=callbacks)

Epoch 1/30
560/560 [==============================] - 70s 116ms/step - loss: 2.4879 - accuracy: 0.1265 - val_loss: 2.3172 - val_accuracy: 0.1739

Epoch 00001: val_accuracy improved from -inf to 0.17387, saving model to full_verse.h5
Epoch 2/30
560/560 [==============================] - 63s 113ms/step - loss: 2.2647 - accuracy: 0.1960 - val_loss: 2.1329 - val_accuracy: 0.2367

Epoch 00002: val_accuracy improved from 0.17387 to 0.23668, saving model to full_verse.h5
Epoch 3/30
560/560 [==============================] - 63s 112ms/step - loss: 1.8096 - accuracy: 0.3603 - val_loss: 1.2206 - val_accuracy: 0.5543

Epoch 00003: val_accuracy improved from 0.23668 to 0.55427, saving model to full_verse.h5
Epoch 4/30
560/560 [==============================] - 63s 112ms/step - loss: 1.1962 - accuracy: 0.6111 - val_loss: 1.1215 - val_accuracy: 0.6477

Epoch 00004: val_accuracy improved from 0.55427 to 0.64774, saving model to full_verse.h5
Epoch 5/30
560/560 [==============================] - 63s 1

In [ ]:
model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 30, batch_size= 32, shuffle = True, callbacks=callbacks)

Epoch 1/30
560/560 [==============================] - 69s 116ms/step - loss: 2.5330 - accuracy: 0.1173 - val_loss: 2.3200 - val_accuracy: 0.1774

Epoch 00001: val_accuracy did not improve from 0.94623
Epoch 2/30
560/560 [==============================] - 63s 112ms/step - loss: 2.3548 - accuracy: 0.1711 - val_loss: 2.2481 - val_accuracy: 0.2090

Epoch 00002: val_accuracy did not improve from 0.94623
Epoch 3/30
560/560 [==============================] - 63s 112ms/step - loss: 2.2656 - accuracy: 0.1823 - val_loss: 2.1516 - val_accuracy: 0.2136

Epoch 00003: val_accuracy did not improve from 0.94623
Epoch 4/30
560/560 [==============================] - 63s 112ms/step - loss: 2.1104 - accuracy: 0.2237 - val_loss: 1.9447 - val_accuracy: 0.2915

Epoch 00004: val_accuracy did not improve from 0.94623
Epoch 5/30
560/560 [==============================] - 63s 113ms/step - loss: 1.9028 - accuracy: 0.2942 - val_loss: 1.7354 - val_accuracy: 0.3678

Epoch 00005: val_accuracy did not improve from 0.9

## after modelig

In [ ]:
## Loading best weights

model = tf.keras.models.load_model('full_verse.h5')

## Classify on the test set splitted from the dataset 

In [ ]:
def classify(sentence):

  sequence = [char2idx[char] for char in sentence]
  sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)

  pred = model.predict(sequence)[0]
  pred = np.argmax(pred, 0)
  return pred

In [ ]:
prediction = []
for i in tqdm(range(len(x_test))):
    prediction.append(classify(x_test[i]))

100%|██████████| 100/100 [00:07<00:00, 14.28it/s]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        10
           1       0.80      1.00      0.89         8
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00         3
           4       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00        12
           8       1.00      0.90      0.95        10
           9       1.00      1.00      1.00        10
          10       1.00      1.00      1.00         5
          11       0.89      1.00      0.94         8
          12       1.00      1.00      1.00         1
          13       0.89      0.80      0.84        10

    accuracy                           0.95       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.95      0.95      0.95       100



## Concatenate  to get ' High results '
Concatenate the training data with the predictions made in order to get a higher generalization of the process

In [ ]:
X_test = to_sequences(x_test)
TEST = to_sequences(test)
Preds = np.array(preds)
Y_test = np.array(y_test)
X_total = np.concatenate((X_train,X_test,TEST),axis=0)
y_total = np.concatenate((y_train,Y_test,Preds),axis=0)

In [ ]:
callbacks2 = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
callbacks2 += [tf.keras.callbacks.ModelCheckpoint('full_verse2.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')]

model.fit(X_total, y_total, validation_data= (X_valid, y_valid), epochs =6 , batch_size= 32, shuffle = True,callbacks= callbacks2)
model = tf.keras.models.load_model('full_verse2.h5')

Epoch 1/6
626/626 [==============================] - 74s 115ms/step - loss: 0.0942 - accuracy: 0.9748 - val_loss: 0.3772 - val_accuracy: 0.9538

Epoch 00001: val_accuracy improved from -inf to 0.95377, saving model to full_verse2.h5
Epoch 2/6
626/626 [==============================] - 71s 113ms/step - loss: 0.0926 - accuracy: 0.9756 - val_loss: 0.3740 - val_accuracy: 0.9528

Epoch 00002: val_accuracy did not improve from 0.95377
Epoch 3/6
626/626 [==============================] - 70s 112ms/step - loss: 0.0886 - accuracy: 0.9758 - val_loss: 0.3803 - val_accuracy: 0.9533

Epoch 00003: val_accuracy did not improve from 0.95377
Epoch 4/6
626/626 [==============================] - 70s 112ms/step - loss: 0.0914 - accuracy: 0.9776 - val_loss: 0.3764 - val_accuracy: 0.9528

Epoch 00004: val_accuracy did not improve from 0.95377
Epoch 5/6
626/626 [==============================] - 70s 112ms/step - loss: 0.0891 - accuracy: 0.9765 - val_loss: 0.3895 - val_accuracy: 0.9543

Epoch 00005: val_accur

In [ ]:
model = tf.keras.models.load_model('full_verse2.h5')

## Submission
Submit the predictions

In [ ]:
submission = pd.read_csv('test.csv')
submission

,id,data
0,0,بلوعة قذف للنية القذف # أخي سنادي معيني قوتي جدتي
1,1,وعَلِيقِي عليه أَرْخَصُ مِنْ ما # لِ المَوارِي...
2,2,تنزه في الثغر مبني في # محل العذيب وفي بارق
3,3,قديرٌ ذو قُدورٍ راسياتٍ # تُقابلُها جِفانٌ كال...
4,4,وبانبي الهدى وافتك من بعد # مديحة من كثير العي...
...,...,...
1995,1995,وَتَنفي الضَفادِعَ أَنفاسُها # فَهُنَّ فُوَيقَ...
1996,1996,هزّ عِطفَيْهِ النّدى فانتزعا # ما أعارَتْهُ ال...
1997,1997,لَكِن عَدَتني نائِباتُ النَوى # في دَوحِهِ وَا...
1998,1998,داجِنٌ بِالطِرادِ يَرمي بِطَرفٍ # في فَضاءٍ وَ...


In [ ]:
test = np.array(submission['data'])
test

array(['بلوعة قذف للنية القذف # أخي سنادي معيني قوتي جدتي',
       'وعَلِيقِي عليه أَرْخَصُ مِنْ ما # لِ المَوارِيثِ في شِرَا ابنِ جُبَارَهْ',
       'تنزه في الثغر مبني في # محل العذيب وفي بارق', ...,
       'لَكِن عَدَتني نائِباتُ النَوى # في دَوحِهِ وَالشادنُ الأَحوَرُ',
       'داجِنٌ بِالطِرادِ يَرمي بِطَرفٍ # في فَضاءٍ وَفي صَحارٍ بَساطِ',
       'لا تَسَلْ عن عِداكَ أين اسْتَقَرّوا # لَحِقَ القَوْمُ باللّطيفِ الخَبيرِ'],
      dtype=object)

In [ ]:
for i in tqdm(range(len(test))):
  out = ""
  #test[i] = araby.strip_tashkeel(test[i])
  for c in test[i]:
    if c not in execluded:
      out += c
  test[i] = out

100%|██████████| 2000/2000 [00:00<00:00, 58620.19it/s]


In [ ]:
preds = []
for i in tqdm(range(len(test))):
    preds.append(classify(test[i]))

100%|██████████| 2000/2000 [02:01<00:00, 16.51it/s]


In [ ]:
ids = [str(id_val) for id_val in range(submission.shape[0])]
sample_submission = pd.DataFrame(np.array([ids,preds]).T,columns=['id','labels'])
sample_submission.to_csv("sample_submission13_95xxx.csv",index=False)

In [ ]:
sample_submission

,id,labels
0,0,8
1,1,9
2,2,2
3,3,11
4,4,8
...,...,...
1995,1995,2
1996,1996,7
1997,1997,0
1998,1998,9


## Final submission

In [ ]:
submission_final = pd.read_csv('/content/final_round_test.csv')
submission_final

,id,data
0,0,أَمّا الوِلايَةُ فَالمَعروفُ خُطبَتُها # وَلَس...
1,1,إِذا مامَرَرتَ بِأَهلِ القُبورِ # تَيَقَّنتَ أ...
2,2,فعِشْ إنْ قدرتَ قليلَ الحديثِ # قليل الجَليس ق...
3,3,لَخُطّابُ لَيلى بالَ بُرثُنَ مِنكُمُ # أَذَلُّ...
4,4,وَخَطفُكَ مِن يَدٍ تَهمي وَتَحمي # فَتَختَصِمُ...
...,...,...
1995,1995,أَخُطُّ وَأَمحو كُلَّ ما قَد خَطَطتُهُ # بِدَم...
1996,1996,فَاِعرَفي لِلمَشيبِ إِذ شَمِلَ الرَأ # سَ فَإِ...
1997,1997,إِنَّ الدِيارَ الَّتي تُبكى بِمُتَّقِدٍ # غَير...
1998,1998,فاخترت هذا ولا العشار من كضض # تركت قلبي كريش ...


In [ ]:
final_test = np.array(submission_final['data'])
final_test

array(['أَمّا الوِلايَةُ فَالمَعروفُ خُطبَتُها # وَلَستُ أَقبَلُها عَن بابِكُم بَدَلا',
       'إِذا مامَرَرتَ بِأَهلِ القُبورِ # تَيَقَّنتَ أَنَّكَ مِنهُم غَدا',
       'فعِشْ إنْ قدرتَ قليلَ الحديثِ # قليل الجَليس قليل الخِصام', ...,
       'إِنَّ الدِيارَ الَّتي تُبكى بِمُتَّقِدٍ # غَيرُ الدِيارِ الَّتي تَبكي بِهَطّالِ',
       'فاخترت هذا ولا العشار من كضض # تركت قلبي كريش النسر منتفضا',
       'وَتَرجِعُ لي روحُ الحَياةِ فَإِنَّني # بِنَفسِيَ لَو عايَنتَني لَأَجودُ'],
      dtype=object)

In [ ]:
for i in tqdm(range(len(final_test))):
  out = ""
  #test[i] = araby.strip_tashkeel(test[i])
  for c in final_test[i]:
    if c not in execluded:
      out += c
  final_test[i] = out

100%|██████████| 2000/2000 [00:00<00:00, 53892.66it/s]


In [ ]:
final_preds = []
for i in tqdm(range(len(final_test))):
    final_preds.append(classify(final_test[i]))

100%|██████████| 2000/2000 [02:01<00:00, 16.45it/s]


In [ ]:
ids = [str(id_val) for id_val in range(submission_final.shape[0])]
sample_submission = pd.DataFrame(np.array([ids,final_preds]).T,columns=['id','labels'])
sample_submission.to_csv("final_submission_v9530.csv",index=False)

In [ ]:
sample_submission

,id,labels
0,0,8
1,1,9
2,2,2
3,3,11
4,4,8
...,...,...
1995,1995,2
1996,1996,7
1997,1997,0
1998,1998,9
